In [2]:
from os_helper import OsHelper
import pandas as pd

os_helper = OsHelper()

In [4]:
from compute_metrics import ComputeMetrics

price_data = os_helper.read_data(directory_name="base data", file_name="close_fixed.csv", index_col=0, low_memory=False)
trade_volume = os_helper.read_data(directory_name="base data", file_name="volume_fixed.csv", index_col=0, low_memory=False)
spy = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0, low_memory=False)

print(price_data.head())
print(trade_volume.head())
print(spy.head())

rolling_window_years = [1, 2, 3]

# Compute all risk metrics needed for the strategy and the filter
metrics_calculator = ComputeMetrics(price_data=price_data, trade_volume=trade_volume, rolling_window_years=rolling_window_years,
                                    benchmark_series=spy['Adj Close'])

final_df = metrics_calculator.compile_all_metrics()

os_helper.write_data(directory_name="transform data", file_name="all_metrics.csv", data_frame=final_df)

Loaded file close_fixed.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file volume_fixed.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
Loaded file SPY (1).csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
                  Open        High         Low       Close   Adj Close     Volume
Date                                                                             
1995-01-03   45.703125   45.843750   45.687500   45.781250   27.352005     324300
1995-01-04   45.984375   46.000000   45.750000   46.000000   27.482687     351800
1995-01-05   46.031250   46.109375   45.953125   46.000000   27.482687      89800
1995-01-06   46.093750   46.250000   45.906250   46.046875   27.510689     448400
1995-01-09   46.031250   46.093750   46.000000   46.093750   27.538696      36800
...      

In [5]:
from get_rebal_dates import GetRebalDates, filter_by_rebalance_dates

all_metrics = os_helper.read_data(directory_name="transform data", file_name="all_metrics.csv", index_col=0, header=[0, 1])
print(all_metrics.head())

termination_date = '2015-12-31' # Last day of the backtest
initial_year = 2000 # First year of the backtest
reb_month = 1
reb_week = 2
reb_weekday = 'FRI'
reb_frequency = 'Q' # Q for quarterly, M for monthly, A for annual, S for semi-annual

# Get the rebalance dates of the strategy
get_reb_dates = GetRebalDates(termination_date=termination_date, initial_year=initial_year, reb_month=reb_month,
                              reb_week=reb_week, reb_weekday=reb_weekday, reb_frequency=reb_frequency)
rebalance_dates = get_reb_dates.reb_dates

print("Dates de rebalancement :")
for date in rebalance_dates:
    print(date)

reb_metrics = filter_by_rebalance_dates(get_reb_dates=get_reb_dates, data=all_metrics)
print(reb_metrics.head())

os_helper.write_data(directory_name="transform data", file_name=f"rebalance_metrics.csv", data_frame=reb_metrics)

Loaded file all_metrics.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
                   A US Equity                                                                                                                AAPL US Equity                                                                                                                 ABT US Equity                      ... YUM US Equity                     ZBH US Equity                                                                                                                ZION US Equity                                                                                                          
           average_turnover_1Y average_turnover_2Y average_turnover_3Y beta_1Y beta_2Y beta_3Y volatility_1Y volatility_2Y volatility_3Y average_turnover_1Y average_turnover_2Y average_turnover_3Y beta_1Y beta_2Y beta_3Y volatility_1Y volatility_2Y volatility_3Y aver

In [6]:
from filter import Filter

data = os_helper.read_data(directory_name="transform data", file_name="rebalance_metrics.csv", index_col=0)
print(data)
risk_filter = Filter(data=data)

risk_top_n = 120
liquidity_top_n = 200

filtered_data = risk_filter.apply_filters(liquidity_top_n=liquidity_top_n, risk_top_n=risk_top_n)
print(filtered_data.head())

os_helper.write_data(directory_name="transform data", file_name="filtered_data.csv", data_frame=filtered_data)

Loaded file rebalance_metrics.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
             Date          Ticker  average_turnover_1Y  average_turnover_2Y  average_turnover_3Y   beta_1Y   beta_2Y   beta_3Y  volatility_1Y  volatility_2Y  volatility_3Y
0      2000-01-07  AAPL US Equity         2.822167e+08         2.080641e+08         1.553083e+08  1.351092  1.165324  0.983969       0.602578       0.574981       0.618927
1      2000-01-07   ABT US Equity         6.775778e+07         5.411452e+07         4.707382e+07  0.932067  0.841249  0.866634       0.361892       0.323613       0.305512
2      2000-01-07  ADBE US Equity         8.940289e+07         6.678857e+07         6.509780e+07  0.966219  0.927739  0.872973       0.496930       0.521665       0.515052
3      2000-01-07   ADI US Equity         6.310535e+07         4.873519e+07         4.258878e+07  1.244670  1.140282  1.220119       0.604444       0.623926     

In [6]:
from weighting import InverseMetricsWeighting

filtered_data = os_helper.read_data(directory_name="transform data", file_name="filtered_data.csv", index_col=0)
print(filtered_data)

years_to_inverse = [1, 2, 3]
metric_to_inverse = "beta"
weighting_list = [] # Equal weighting of betas
min_weight = 0.00001
max_weight = 0.05

inverse_metrics_weighting = InverseMetricsWeighting(filtered_data=filtered_data, years_list=years_to_inverse,
                                                    target_metric=metric_to_inverse, weight_list=weighting_list,
                                                    min_limit=min_weight, max_limit=max_weight)
inverse_metrics_weighting_df = inverse_metrics_weighting.compute_inverse_weighted_metrics()
print(inverse_metrics_weighting_df)

os_helper.write_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", data_frame=inverse_metrics_weighting_df)

Loaded file filtered_data.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/transform data.
            Date          Ticker  average_turnover_1Y  average_turnover_2Y  average_turnover_3Y   beta_1Y   beta_2Y   beta_3Y  volatility_1Y  volatility_2Y  volatility_3Y
0     2000-01-07   ADM US Equity         1.685440e+07         1.982424e+07         2.319849e+07  0.376382  0.464531  0.516650       0.321483       0.331964       0.319116
1     2000-01-07   AEP US Equity         1.684560e+07         1.810501e+07         1.777850e+07  0.275702  0.248073  0.278439       0.209497       0.197221       0.180720
2     2000-01-07   APA US Equity         3.053278e+07         2.260925e+07         2.102268e+07  0.266976  0.549218  0.480619       0.511574       0.476057       0.426099
3     2000-01-07   APD US Equity         2.609587e+07         2.336744e+07         2.150346e+07  0.535894  0.619094  0.576415       0.455714       0.390075       0.34531

In [7]:
from portfolio_returns import PortfolioReturns

prices = os_helper.read_data(directory_name="base data", file_name="tot_retl.csv", index_col=0, sep=',')
print(prices.head())
weighting_df = os_helper.read_data(directory_name="transform data", file_name="inverse_metrics_weighting.csv", index_col=0)
print(weighting_df.head())

rebalance_lag = 7 # delay between weight computation and implementation
transaction_fee = 0.001 # 0.1% transaction fee

pf_returns = PortfolioReturns(prices=prices, weighting_data=weighting_df, rebalance_lag=rebalance_lag, transaction_fee=transaction_fee)
drifted_weights = pf_returns.drifted_weights
print(drifted_weights.head())
portfolio_returns = pf_returns.portfolio_returns
print(portfolio_returns.head())

os_helper.write_data(directory_name="final data", file_name="portfolio_returns.csv", data_frame=portfolio_returns)
os_helper.write_data(directory_name="final data", file_name="drifted_weights.csv", data_frame=drifted_weights)

Loaded file tot_retl.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/base data.
            CSCO US Equity  ISRG US Equity  TPR US Equity  DVN US Equity  MRO US Equity  BA US Equity  GILD US Equity  MDT US Equity  V US Equity  A US Equity  MO US Equity  CTRA US Equity  MCHP US Equity  EIX US Equity  BBY US Equity  GEN US Equity  WBA US Equity  DTE US Equity  C US Equity  T US Equity  ...  WYNN US Equity  RHI US Equity  EFX US Equity  NUE US Equity  CTSH US Equity  SWK US Equity  MU US Equity  TRV US Equity  L US Equity  AEP US Equity  CI US Equity  BALL US Equity  JNJ US Equity  DOV US Equity  CRM US Equity  PGR US Equity  WAT US Equity  BWA US Equity  BLK US Equity  PPL US Equity
Date                                                                                                                                                                                                                                                         

In [8]:
from perf_report import PerfReport

benchmark_prices = os_helper.read_data(directory_name="benchmark", file_name="SPY (1).csv", index_col=0)
benchmark_prices.sort_index(inplace=True)
benchmark_prices.index.name = None
benchmark_prices = benchmark_prices[["Adj Close"]]
benchmark_prices.columns = ["SPY"]

print(benchmark_prices.head())

strategy_returns = os_helper.read_data(directory_name="final data", file_name="portfolio_returns.csv", index_col=0)
print(strategy_returns.head())
strategy_returns.columns = ["Portfolio_Returns"]

perf_report = PerfReport(port_returns=strategy_returns, bench_returns=benchmark_prices, strat_name="InverseMetrics")
print(perf_report.get_performance_indicators(rf=0.01, periods_per_year=252))
perf_report.generate_html_report(rf=0.01, periods_per_year=252)

Loaded file SPY (1).csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/benchmark.
                  SPY
1995-01-03  27.352005
1995-01-04  27.482687
1995-01-05  27.482687
1995-01-06  27.510689
1995-01-09  27.538696
Loaded file portfolio_returns.csv from folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Data/final data.
            Portfolio_Returns
2000-01-14           0.000000
2000-01-18           0.008634
2000-01-19          -0.000263
2000-01-20          -0.005856
2000-01-21          -0.008694
{'Cumulative Returns': 11.877385519279322, 'CAGR': 0.17350031146237654, 'Sharpe': 2.168397777292199, 'Sortino': 3.3934477358648563, 'Calmar': 2.293939827530151, 'Max Drawdown': -0.07563420338238847, 'Average Return': 0.0006465487622642554, 'Volatility': 0.07053096627334766, 'Tail Ratio': 1.1888834448753727, 'Gain to Pain Ratio': 0.4786643759346056, 'Value at Risk (VaR)': -0.006661751586426146, 'Conditi

/Users/thomasvaudescal/.virtualenvs/PolyHacks/lib/python3.11/site-packages/quantstats/stats.py:968: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  returns = returns.pivot('Year', 'Month', 'Returns').fillna(0)


Moved file InverseMetrics_backtesting_report.html to folder /Users/thomasvaudescal/Documents/Mes_documents/30_TradingClub/PolyFinance/PolyHacks/Codes/reports.
